# Analyze BabyEnv performance

In [1]:
import gym
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time

import baby.envs
from baby.envs.baby_env import default_conf

In [2]:
# Create env
env = gym.make('baby-v0')

In [3]:
n_measure=100

In [4]:
t=time.time()
for _ in range(n_measure):
    env.reset()

dt = time.time() - t

print(f"Reset duration {dt/n_measure}")

Reset duration 0.29923344135284424


In [5]:
t=time.time()
for _ in range(n_measure):
    env.step([0])
dt_step = time.time() - t

print(f"Step duration {dt_step/n_measure}")

Step duration 0.004349322319030762


In [6]:
print(f"So it requires {dt_step/n_measure*1e6/3600} hours to make 1M steps")

So it requires 1.208145088619656 hours to make 1M steps


In [7]:
t=time.time()
for _ in range(n_measure):
    env.predict(t=0)
dt = time.time() - t

print(f"Predict function duration {dt/n_measure}")
print(f"Predict takes {dt/dt_step*100}% of step function duration")

Predict function duration 0.004241015911102295
Predict takes 97.50980957528567% of step function duration


In [8]:
from scipy.ndimage.filters import uniform_filter

In [9]:
t=time.time()
for _ in range(n_measure):
    for _ in range(env.conf['n_frame']):
        uniform_filter(env.ground_truth[:,:,0])

dt = time.time() - t
print(f"{env.conf['n_frame']} calls to uniform filter call lasts for {dt/n_measure}")
print(f"Those calls take {dt/dt_step*100}% of step function duration")

20 calls to uniform filter call lasts for 0.0013467144966125488
Those calls take 30.963777752683587% of step function duration


In [10]:
t=time.time()
for _ in range(n_measure):
    for _ in range(env.conf['n_frame']):
        np.exp(env.ground_truth[:,:,0])

dt = time.time() - t
print(f"{env.conf['n_frame']} calls to np.exp call lasts for {dt/n_measure}")
print(f"Those calls take {dt/dt_step*100}% of step function duration")

20 calls to np.exp call lasts for 0.00022372961044311523
Those calls take 5.144010814378388% of step function duration


In [11]:
t=time.time()
for _ in range(n_measure):
    for _ in range(env.conf['n_frame']):
        np.clip(env.ground_truth[:,:,0], 0, 1)

dt = time.time() - t
print(f"{env.conf['n_frame']} calls to np.clip call lasts for {dt/n_measure}")
print(f"Those calls take {dt/dt_step*100}% of step function duration")

20 calls to np.clip call lasts for 0.000738217830657959
Those calls take 16.973169071140937% of step function duration


In [13]:
t=time.time()
for _ in range(n_measure):
    for _ in range(env.conf['n_frame']):
        np.copy(env.ground_truth[:,:,0])

dt = time.time() - t
print(f"{env.conf['n_frame']} calls to np.copy call lasts for {dt/n_measure}")
print(f"Those calls take {dt/dt_step*100}% of step function duration")

20 calls to np.copy call lasts for 0.00010972738265991211
Those calls take 2.5228616002955757% of step function duration


In [19]:
# Single clip vs multiple clip
t = time.time()
for _ in range(n_measure):
    for i in range(env.conf['n_frame']):
        np.clip(env.ground_truth[:,:,i], 0, 1)
dt1 = time.time()-t


t = time.time()
for _ in range(n_measure):
    np.clip(env.ground_truth[:,:,0:20], 0, 1)
dt2 = time.time()-t

print(f"Multiple {dt1}s // Single {dt2}s // {dt1/dt2} slower")

Multiple 0.08451342582702637s // Single 0.005934000015258789s // 14.24223552573426 slower
